In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMClassifier
import os

In [5]:
#global variables
KAGGLE = 1
PATH = "/kaggle/input/" if KAGGLE ==1 else "C:/Users/pzlpa_000/"

In [6]:
df_train = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/train-metadata.csv"))
df_test = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/test-metadata.csv"))

/tmp/ipykernel_33/3060065471.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv( os.path.join(PATH,"isic-2024-challenge/train-metadata.csv"))


In [4]:
data = {
    "field_name": [
        "target", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", 
        "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence", 
        "isic_id", "patient_id", "age_approx", "sex", "anatom_site_general", 
        "clin_size_long_diam_mm", "image_type", "tbp_tile_type", "tbp_lv_A", "tbp_lv_Aext", 
        "tbp_lv_B", "tbp_lv_Bext", "tbp_lv_C", "tbp_lv_Cext", "tbp_lv_H", "tbp_lv_Hext", 
        "tbp_lv_L", "tbp_lv_Lext", "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio", 
        "tbp_lv_color_std_mean", "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", 
        "tbp_lv_deltaLBnorm", "tbp_lv_eccentricity", "tbp_lv_location", 
        "tbp_lv_location_simple", "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", 
        "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_perimeterMM", 
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt", 
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle", "tbp_lv_x", "tbp_lv_y", "tbp_lv_z", 
        "attribution", "copyright_license"
    ],
    "description": [
        "Binary class {0: benign, 1: malignant}.", 
        "Unique lesion identifier. Present in lesions that were manually tagged as a lesion of interest.", 
        "Fully classified lesion diagnosis.", 
        "First level lesion diagnosis.", 
        "Second level lesion diagnosis.", 
        "Third level lesion diagnosis.", 
        "Fourth level lesion diagnosis.", 
        "Fifth level lesion diagnosis.", 
        "Mitotic index of invasive malignant melanomas.", 
        "Thickness in depth of melanoma invasion.", 
        "Lesion confidence score (0-100 scale).", 
        "Unique case identifier.", 
        "Unique patient identifier.", 
        "Approximate age of patient at time of imaging.", 
        "Sex of the person.", 
        "Location of the lesion on the patient's body.", 
        "Maximum diameter of the lesion (mm).", 
        "Structured field of the ISIC Archive for image type.", 
        "Lighting modality of the 3D TBP source image.", 
        "A inside lesion.", 
        "A outside lesion.", 
        "B inside lesion.", 
        "B outside lesion.", 
        "Chroma inside lesion.", 
        "Chroma outside lesion.", 
        "Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).", 
        "Hue outside lesion.", 
        "L inside lesion.", 
        "L outside lesion.", 
        "Area of lesion (mm^2).", 
        "Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.", 
        "Color irregularity, calculated as the variance of colors within the lesion's boundary.", 
        "Average A contrast (inside vs. outside lesion).", 
        "Average B contrast (inside vs. outside lesion).", 
        "Average L contrast (inside vs. outside lesion).", 
        "Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.", 
        "Eccentricity.", 
        "Classification of anatomical location, divides arms & legs to upper & lower; torso into thirds.", 
        "Classification of anatomical location, simple.", 
        "Smallest lesion diameter (mm).", 
        "Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.", 
        "Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.", 
        "Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.", 
        "Perimeter of lesion (mm).", 
        "Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.", 
        "Standard deviation of L inside lesion.", 
        "Standard deviation of L outside lesion.", 
        "Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.", 
        "Lesion border asymmetry angle.", 
        "X-coordinate of the lesion on 3D TBP.", 
        "Y-coordinate of the lesion on 3D TBP.", 
        "Z-coordinate of the lesion on 3D TBP.", 
        "Image attribution, synonymous with image source.", 
        "Copyright license."
    ],
    "train_only": [
        True, True, True, True, True, True, True, True, True, True, True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False
    ]
}

df_explained = pd.DataFrame(data)


In [82]:
print(df_explained['description'][df_explained['field_name']=="tbp_lv_color_std_mean"])

31    Color irregularity, calculated as the variance...
Name: description, dtype: object


In [70]:
df_train[["tbp_lv_color_std_mean","tbp_lv_radial_color_std_max"]].describe()
#df_train[["tbp_lv_color_std_mean","tbp_lv_radial_color_std_max","target"]][(df_train["tbp_lv_radial_color_std_max"]==0)&(df_train["tbp_lv_color_std_mean"]!=0)&(df_train['target']==1)]

,tbp_lv_color_std_mean,tbp_lv_radial_color_std_max
count,401059.000000,401059.000000
mean,1.070408,1.016459
std,0.762918,0.734631
min,0.000000,0.000000
25%,0.574909,0.563891
50%,0.931402,0.902281
75%,1.412154,1.334523
max,9.952932,11.491140


In [72]:
df_train[["tbp_lv_color_std_mean","tbp_lv_radial_color_std_max"]][(df_train['target']==1)]

,tbp_lv_color_std_mean,tbp_lv_radial_color_std_max
387,3.287692,3.514434
935,0.624458,0.549971
1245,1.401430,1.340916
1846,1.472167,1.104453
3478,2.243673,2.904396
...,...,...
396165,2.179194,2.432117
398671,0.907498,0.814219
399936,3.895505,4.097754
399943,2.291503,2.283022


In [6]:
test_and_target=["target", 
        "isic_id", "patient_id", "age_approx", "sex", "anatom_site_general", 
        "clin_size_long_diam_mm", "image_type", "tbp_tile_type", "tbp_lv_A", "tbp_lv_Aext", 
        "tbp_lv_B", "tbp_lv_Bext", "tbp_lv_C", "tbp_lv_Cext", "tbp_lv_H", "tbp_lv_Hext", 
        "tbp_lv_L", "tbp_lv_Lext", "tbp_lv_areaMM2", "tbp_lv_area_perim_ratio", 
        "tbp_lv_color_std_mean", "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", 
        "tbp_lv_deltaLBnorm", "tbp_lv_eccentricity", "tbp_lv_location", 
        "tbp_lv_location_simple", "tbp_lv_minorAxisMM", "tbp_lv_nevi_confidence", 
        "tbp_lv_norm_border", "tbp_lv_norm_color", "tbp_lv_perimeterMM", 
        "tbp_lv_radial_color_std_max", "tbp_lv_stdL", "tbp_lv_stdLExt", 
        "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle", "tbp_lv_x", "tbp_lv_y", "tbp_lv_z", 
        "attribution", "copyright_license"
    ]

In [19]:
df_train_filtered=df_train[test_and_target]
categorical_columns = df_train_filtered.select_dtypes(include=['object', 'category']).columns
numerical_columns = df_train_filtered.select_dtypes(include=['number']).columns

In [9]:
def value_counts (df, value):
    value_columns = df.columns[(df == value).any()]
    print(value_columns)

    # Count number of 0s in each column that has at least one 0
    value_counts = df[value_columns].eq(0).sum()

    # Print the number of 0s in each relevant column
    print(f"Number of {value} in each column that has at least one {value}:{value_counts}")

value_counts(df_train_filtered,0)
 

Index(['target', 'tbp_lv_color_std_mean', 'tbp_lv_nevi_confidence',
       'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max',
       'tbp_lv_symm_2axis_angle'],
      dtype='object')
Number of 0 in each column that has at least one 0:target                         400666
tbp_lv_color_std_mean           26187
tbp_lv_nevi_confidence              6
tbp_lv_norm_color               26182
tbp_lv_radial_color_std_max     29733
tbp_lv_symm_2axis_angle         18562
dtype: int64


In [84]:
df_train[["tbp_lv_color_std_mean","tbp_lv_radial_color_std_max","tbp_lv_symm_2axis_angle"]][(df_train["tbp_lv_radial_color_std_max"]==0) | (df_train["tbp_lv_symm_2axis_angle"]==0)]

,tbp_lv_color_std_mean,tbp_lv_radial_color_std_max,tbp_lv_symm_2axis_angle
0,0.000000,0.00000,85
1,0.000000,0.00000,55
2,0.000000,0.00000,105
4,0.000000,0.00000,20
5,0.000000,0.00000,55
...,...,...,...
401005,0.000000,0.00000,15
401006,0.000000,0.00000,105
401008,0.000000,0.00000,40
401020,1.331853,1.91062,0


In [31]:
# Print the identified categorical and numerical columns
print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

# Calculate mean for numerical columns grouped by 'target'
means_by_target = df_train_filtered[numerical_columns].groupby(df_train_filtered['target']).mean()

print(means_by_target)

for c in categorical_columns:
    num_dif = df_train_filtered[c].nunique()
    if num_dif < 30:
        print (f'{c}: {num_dif} different values : {df_train_filtered[c].unique()}')
    else:
        print (f'{c}: {num_dif} different values')             
    

Categorical columns: Index(['isic_id', 'patient_id', 'sex', 'anatom_site_general', 'image_type',
       'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple',
       'attribution', 'copyright_license'],
      dtype='object')
Numerical columns: Index(['target', 'age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A',
       'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext',
       'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA',
       'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence',
       'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z'],
      dtype='object')
        target  age_approx  clin_size_long_d

In [10]:

place_by_target = df_train_filtered.groupby(['anatom_site_general', 'target']).size().unstack(fill_value=0)
place_totals = place_by_target.sum(axis=1)
place_by_target['percentage_1'] = (place_by_target[1] / place_totals) * 100
print(place_by_target)

target                    0    1  percentage_1
anatom_site_general                           
anterior torso        87688   82      0.093426
head/neck             11968   78      0.647518
lower extremity      102955   73      0.070855
posterior torso      121799  103      0.084494
upper extremity       70500   57      0.080786


In [13]:
def feature_engineering(df):
    # Taken from https://www.kaggle.com/code/snnclsr/tabular-ensemble-lgbm-catboost
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
    df["hue_contrast"] = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    df["3d_position_distance"] = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"] 
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"] = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    df["log_lesion_area"] = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"] = np.arctan2(df_train["tbp_lv_y"], df_train["tbp_lv_x"])
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4
    df["color_variance_ratio"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    df["size_color_contrast_ratio"] = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    df["3d_volume_approximation"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    df["color_range"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs() + (df["tbp_lv_A"] - df["tbp_lv_Aext"]).abs() + (df["tbp_lv_B"] - df["tbp_lv_Bext"]).abs()
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    df["border_length_ratio"] = df["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(df["tbp_lv_areaMM2"] / np.pi))
    df["age_size_symmetry_index"] = df["age_approx"] * df["clin_size_long_diam_mm"] * df["tbp_lv_symm_2axis"]   
    return df

In [20]:
df_train_filtered=df_train_filtered[df_train_filtered['anatom_site_general'].notna()]
df_train_filtered[numerical_columns]=df_train_filtered[numerical_columns].fillna(df_train_filtered[numerical_columns].median())
df_train_filtered['tbp_lv_symm_2axis_angle']=df_train_filtered['tbp_lv_symm_2axis_angle'].replace(0, df_train_filtered['tbp_lv_symm_2axis_angle'].median())
df_train_filtered['sex']=df_train_filtered['sex'].fillna('male')
df_train_filtered[['tbp_lv_color_std_mean', 'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max']] = df_train_filtered[['tbp_lv_color_std_mean', 'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max']].replace(0, np.nan)
df_train_filtered=feature_engineering(df_train_filtered)
numerical_columns = df_train_filtered.select_dtypes(include=['number']).columns
null_counts = df_train_filtered.isnull().sum()
null_columns = null_counts[null_counts > 0]
null_columns

tbp_lv_color_std_mean          25680
tbp_lv_norm_color              25675
tbp_lv_radial_color_std_max    29167
color_uniformity               29172
lesion_visibility_score        25675
hue_color_std_interaction      25680
lesion_severity_index          25675
color_shape_composite_index    25680
comprehensive_lesion_index     25675
color_variance_ratio           25680
border_color_interaction       25675
color_asymmetry_index          29167
shape_color_consistency        25680
dtype: int64

In [21]:
df_train_filtered[numerical_columns]=df_train_filtered[numerical_columns].fillna(df_train_filtered[numerical_columns].median())

In [22]:
df_train_filtered.tail(10)

,target,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,color_variance_ratio,border_color_interaction,size_color_contrast_ratio,age_normalized_nevi_confidence,color_asymmetry_index,3d_volume_approximation,color_range,shape_color_consistency,border_length_ratio,age_size_symmetry_index
401049,0,ISIC_9999817,IP_1117889,55.0,female,anterior torso,3.02,TBP tile: close-up,3D: XP,22.672866,...,0.070995,0.584319,0.526395,2.746599e-06,0.245038,3852.021486,13.089898,0.124878,1.210992,41.113861
401050,0,ISIC_9999852,IP_1117889,55.0,female,posterior torso,3.56,TBP tile: close-up,3D: XP,16.400388,...,0.117564,5.465314,0.641981,8.742552e-03,0.212907,5327.286279,7.569099,0.187438,1.365111,93.238095
401051,0,ISIC_9999854,IP_6158578,70.0,male,posterior torso,3.12,TBP tile: close-up,3D: XP,21.434500,...,0.352897,5.745529,0.461649,4.566919e-01,0.136117,4665.109816,18.738880,0.530606,1.253805,69.052948
401052,0,ISIC_9999883,IP_5024708,85.0,male,anterior torso,7.58,TBP tile: close-up,3D: XP,19.075495,...,4.307793,31.092171,0.854869,4.362939e-15,0.670955,24884.866910,19.121845,4.725250,1.296340,163.939535
401053,0,ISIC_9999919,IP_3026867,65.0,male,anterior torso,9.47,TBP tile: close-up,3D: white,17.954859,...,0.360610,25.464662,1.866469,2.226384e-05,0.389986,44095.315238,12.808032,0.992095,1.773917,267.064021
401054,0,ISIC_9999937,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,1.189206,22.792496,0.503772,1.419462e+00,0.672625,27420.594341,25.534242,1.920693,1.191578,137.525947
401055,0,ISIC_9999951,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,...,0.279077,12.410814,0.547382,3.852603e-03,0.482148,6699.032644,15.059800,0.584609,1.204498,85.836000
401056,0,ISIC_9999960,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,...,0.225912,2.398463,0.261393,9.222766e-01,0.111242,2957.102296,16.740260,0.323736,1.019062,24.397887
401057,0,ISIC_9999964,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,...,0.673362,5.094737,0.328814,3.310644e+00,0.174507,5306.543645,25.688469,0.649619,1.047466,13.595375
401058,0,ISIC_9999967,IP_6426047,50.0,male,lower extremity,3.30,TBP tile: close-up,3D: XP,16.792900,...,0.305981,5.840794,0.274225,1.873028e+00,0.215348,2191.378374,17.635240,0.653736,1.087319,32.656256


In [23]:
cat_cols = ["sex", "anatom_site_general"]

# Create and fit the OneHotEncoder
one_hot_encoder = OneHotEncoder(
    sparse=False,        # We want a dense array
    handle_unknown='ignore'  # Ignore unknown categories
)
X_cat = one_hot_encoder.fit_transform(df_train_filtered[cat_cols])

# Get feature names for one-hot encoded columns
encoded_cols = one_hot_encoder.get_feature_names_out(cat_cols)

# Create a DataFrame with the encoded columns
df_encoded = pd.DataFrame(X_cat, columns=encoded_cols)
df_encoded

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,sex_female,sex_male,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
395298,0.0,1.0,1.0,0.0,0.0,0.0,0.0
395299,0.0,1.0,0.0,0.0,0.0,1.0,0.0
395300,1.0,0.0,1.0,0.0,0.0,0.0,0.0
395301,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [29]:
scaler = StandardScaler()
df_train_scaled=df_train_filtered[numerical_columns]
numerical_columns_list=list(numerical_columns)
numerical_columns_list.remove("target")
df_train_scaled[numerical_columns_list] = scaler.fit_transform(df_train_scaled[numerical_columns_list])
df_train_scaled.reset_index(drop=True, inplace=True)
df_train_scaled

/tmp/ipykernel_33/3238007868.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_scaled[numerical_columns_list] = scaler.fit_transform(df_train_scaled[numerical_columns_list])


,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,color_variance_ratio,border_color_interaction,size_color_contrast_ratio,age_normalized_nevi_confidence,color_asymmetry_index,3d_volume_approximation,color_range,shape_color_consistency,border_length_ratio,age_size_symmetry_index
0,0,0.150526,-0.511007,0.070754,0.386602,-0.253092,-0.656013,-0.188295,-0.402430,-0.287850,...,-0.217134,-0.285444,-0.106758,-0.836190,-0.250542,-0.632187,-0.259945,-0.233370,1.626451,0.665352
1,0,0.150526,-1.624706,2.942782,2.976191,-0.365487,-0.522797,1.135301,0.918651,-2.710065,...,-0.217134,-0.285444,-1.323025,-0.836238,-0.250542,-0.682662,-0.256234,-0.233370,-1.488112,-0.971761
2,0,0.150526,-0.304342,0.654621,0.633020,1.846413,1.479545,1.654574,1.400054,0.837925,...,-0.217134,-0.285444,-0.047143,-0.836233,-0.250542,-0.380065,0.251605,-0.233370,1.047813,0.041169
3,0,0.519606,-0.407675,-1.432381,-0.778990,-1.293398,-1.290904,-1.582872,-1.355663,0.320783,...,-0.589334,-0.790397,-0.196287,-0.470313,-0.711458,-0.033268,-1.531147,-0.894436,-0.843793,-0.510950
4,0,-0.218555,-0.688970,1.192979,1.466359,-0.340041,-0.262611,0.256747,0.358563,-1.396425,...,-0.217134,-0.285444,-0.535738,-0.836211,-0.250542,-0.534312,-0.362207,-0.233370,0.232399,-0.452046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395298,0,0.888687,1.647500,0.654247,0.011849,-0.112312,-0.025839,0.166653,-0.047831,-0.700337,...,1.924965,1.410374,-0.193645,0.699136,1.613434,1.606461,1.582416,2.093109,-0.191233,1.214887
395299,0,0.150526,-0.470823,0.003943,0.319718,1.122073,0.925291,0.845938,0.837285,0.912849,...,-0.728905,0.226054,-0.019738,-0.832071,0.783088,-0.219983,-0.183824,-0.426910,-0.109433,0.262423
395300,0,0.519606,-1.079338,-0.658477,-0.722196,0.302356,-0.085680,-0.042737,-0.342159,0.944643,...,-0.883933,-0.916134,-1.160204,0.161351,-0.833797,-0.549805,0.099542,-0.918949,-1.283467,-0.869665
395301,0,-2.063958,-0.648785,0.582681,-1.518659,0.033624,0.029085,0.240652,-0.464356,-0.496984,...,0.420800,-0.608549,-0.891340,2.744750,-0.558008,-0.342720,1.608422,-0.304294,-1.103634,-1.068717


In [44]:
# Concatenate the original DataFrame with the encoded columns
df_train_encoded = pd.concat([df_train_scaled, df_encoded], axis=1)
train_cols=list(df_train_encoded.columns)
train_cols.remove("target")
ids=df_train_filtered[["isic_id", "patient_id"]]
ids.reset_index(drop=True, inplace=True)
df_train_final=pd.concat([df_train_encoded, ids], axis=1)

print(df_train_final)

        target  age_approx  clin_size_long_diam_mm  tbp_lv_A  tbp_lv_Aext  \
0            0    0.150526               -0.511007  0.070754     0.386602   
1            0    0.150526               -1.624706  2.942782     2.976191   
2            0    0.150526               -0.304342  0.654621     0.633020   
3            0    0.519606               -0.407675 -1.432381    -0.778990   
4            0   -0.218555               -0.688970  1.192979     1.466359   
...        ...         ...                     ...       ...          ...   
395298       0    0.888687                1.647500  0.654247     0.011849   
395299       0    0.150526               -0.470823  0.003943     0.319718   
395300       0    0.519606               -1.079338 -0.658477    -0.722196   
395301       0   -2.063958               -0.648785  0.582681    -1.518659   
395302       0   -0.587636               -0.361749 -0.793628    -0.986671   

        tbp_lv_B  tbp_lv_Bext  tbp_lv_C  tbp_lv_Cext  tbp_lv_H  ...  \
0   

In [36]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [46]:
N_SPLITS = 10
gkf = GroupKFold(n_splits=N_SPLITS)

df_train_final = df_train_final.sample(frac=1).reset_index(drop=True)
df_train_final["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train_final, df_train_final["target"], groups=df_train_final["patient_id"])):
    df_train_final.loc[val_idx, "fold"] = idx
df_train_final

,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,sex_female,sex_male,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,isic_id,patient_id,fold
0,0,0.150526,-0.511007,0.070754,0.386602,-0.253092,-0.656013,-0.188295,-0.402430,-0.287850,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,ISIC_0015670,IP_1235828,2
1,0,0.150526,-1.624706,2.942782,2.976191,-0.365487,-0.522797,1.135301,0.918651,-2.710065,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,ISIC_0015845,IP_8170065,8
2,0,0.150526,-0.304342,0.654621,0.633020,1.846413,1.479545,1.654574,1.400054,0.837925,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,ISIC_0015864,IP_6724798,7
3,0,0.519606,-0.407675,-1.432381,-0.778990,-1.293398,-1.290904,-1.582872,-1.355663,0.320783,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,ISIC_0015902,IP_4111386,8
4,0,-0.218555,-0.688970,1.192979,1.466359,-0.340041,-0.262611,0.256747,0.358563,-1.396425,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,ISIC_0024200,IP_8313778,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395298,0,0.888687,1.647500,0.654247,0.011849,-0.112312,-0.025839,0.166653,-0.047831,-0.700337,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,ISIC_9999937,IP_1140263,8
395299,0,0.150526,-0.470823,0.003943,0.319718,1.122073,0.925291,0.845938,0.837285,0.912849,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,ISIC_9999951,IP_5678181,7
395300,0,0.519606,-1.079338,-0.658477,-0.722196,0.302356,-0.085680,-0.042737,-0.342159,0.944643,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,ISIC_9999960,IP_0076153,3
395301,0,-2.063958,-0.648785,0.582681,-1.518659,0.033624,0.029085,0.240652,-0.464356,-0.496984,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,ISIC_9999964,IP_5231513,7


In [47]:
lgb_params = {
    'objective': 'binary',
    # "random_state": 42,
    "n_estimators": 1500,
    'learning_rate': 0.001,
    'bagging_freq': 1,
    'pos_bagging_fraction': 0.75,
    'neg_bagging_fraction': 0.05,
    'feature_fraction': 0.6,
    'lambda_l1': 0.2,
    'lambda_l2': 0.7,
    'num_leaves': 35,
    "min_data_in_leaf": 50,
    "verbosity": -1,
    # "device": "gpu"
    # "extra_trees": True
}

lgb_scores = []
lgb_models = []
for fold in range(N_SPLITS):
    _df_train = df_train_final[df_train_final["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train_final[df_train_final["fold"] == fold].reset_index(drop=True)
    model = LGBMClassifier(**lgb_params)    
    # model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **lgb_params)) for i in range(1)], voting="soft")
    model.fit(_df_train[train_cols], _df_train["target"])
    preds = model.predict_proba(_df_valid[train_cols])[:, 1]
    score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
    lgb_scores.append(score)
    lgb_models.append(model)

fold: 0 - Partial AUC Score: 0.17875
fold: 1 - Partial AUC Score: 0.15827
fold: 2 - Partial AUC Score: 0.15745
fold: 3 - Partial AUC Score: 0.16445
fold: 4 - Partial AUC Score: 0.13074
fold: 5 - Partial AUC Score: 0.18252
fold: 6 - Partial AUC Score: 0.15267
fold: 7 - Partial AUC Score: 0.17178
fold: 8 - Partial AUC Score: 0.10344
fold: 9 - Partial AUC Score: 0.16934


In [ ]:
lgbm_score = np.mean(lgb_scores)
print(f"LGBM Score: {lgbm_score:.5f}")